In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
def sobelEdgeDetector(image):
    #pixels = image.load()
    pixels = image

    width, height = len(pixels[0]), len(pixels)

    xMatrix = [[-1, 0, 1],
               [-2, 0, 2],
               [-1, 0, 1]]
    
    yMatrix = [[-1, -2, -1],
               [ 0,  0,  0],
               [ 1,  2,  1]]
    
    xSobelImage =       [[0 for _ in range(width)] for _ in range(height)]
    ySobelImage =       [[0 for _ in range(width)] for _ in range(height)]
    uniformSobelImage = [[0 for _ in range(width)] for _ in range(height)]

    for y in range(1, height-1):
        for x in range(1, width-1):
            #print(x, ", ", y)
            imageMatrix = image[y-1:y+2, x-1:x+2]
            
            xValue = convolve(xMatrix, imageMatrix)
            yValue = convolve(yMatrix, imageMatrix)

            uniformValue = np.sqrt((xValue**2) + (yValue**2))

            xSobelImage[y][x] = xValue
            ySobelImage[y][x] = yValue
            uniformSobelImage[y][x] = uniformValue
    
    #xSobelImage, ySobelImage = uniformGradient(xSobelImage), uniformGradient(ySobelImage)

    return xSobelImage, ySobelImage, uniformSobelImage

def convolve(matrix, image):
    result = 0
    for i in range(3):
        for j in range(3):
            result += matrix[i][j] * image[i][j]

    return result

def uniformGradient(sobelImage):
    width, height = len(sobelImage[0]), len(sobelImage)

    for row in range(height):
        for col in range(width):
            if sobelImage[row][col] < 20:
                sobelImage[row][col] = 255
            elif sobelImage[row][col] > 20:
                sobelImage[row][col] = 255
            else:
                sobelImage[row][col] = 0

In [3]:
#textImage = "cube/cube.png"
textImage = "rectas/rectas.JPG"

originalImage = cv2.imread(textImage)

bNwImage = cv2.cvtColor(originalImage, cv2.COLOR_BGR2GRAY)

xSobelImage, ySobelImage, uniformImage = sobelEdgeDetector(bNwImage)

In [4]:
xSobelImage = cv2.normalize(np.abs(xSobelImage), None, 0, 255, cv2.NORM_MINMAX)
ySobelImage = cv2.normalize(np.abs(ySobelImage), None, 0, 255, cv2.NORM_MINMAX)
uniformImage = cv2.normalize(np.abs(uniformImage), None, 0, 255, cv2.NORM_MINMAX)

xSobelImage = xSobelImage.astype(np.uint8)
ySobelImage = ySobelImage.astype(np.uint8)
uniformImage = uniformImage.astype(np.uint8)

cv2.imwrite(textImage + 'x_sobel.jpg', xSobelImage)
cv2.imwrite(textImage +'y_sobel.jpg', ySobelImage)
cv2.imwrite(textImage +'uniform_sobel.jpg', uniformImage)

True

In [5]:
def houghMatrixOriginal(image):

    pixels = image
    width, height = len(pixels[0]), len(pixels)

    y_dim = int(np.sqrt((width**2)+(height**2)))
    x_dim = 360

    matriz = np.zeros((x_dim, y_dim))
    
    for y in range(1, height-1):
        for x in range(1, width-1):
            if pixels[y][x] > 200:
                for theta in range(x_dim):
                    theta_rad = np.deg2rad(theta)
                    rho = int(x * np.cos(theta_rad) + y * np.sin(theta_rad))
                    if rho < np.sqrt((width**2)+(height**2)):
                        matriz[theta][rho] += 1
    
    n_mayores = 10

    indices_aplanados = np.argpartition(matriz.flatten(), -n_mayores)[-n_mayores:]
    valores_mayores = matriz.flatten()[indices_aplanados]
    indices_mayores = np.unravel_index(indices_aplanados, matriz.shape)

    orden = np.argsort(-valores_mayores)
    valores_mayores = valores_mayores[orden]
    indices_mayores = tuple(indice[orden] for indice in indices_mayores)

    lista = []

    for i in range(len(indices_mayores[0])):
        lista.append((indices_mayores[0][i], indices_mayores[1][i]))

    return lista


In [6]:
def houghMatrix(image):
    height, width = image.shape
    diagonal = int(np.sqrt(height**2 + width**2))
    thetas = np.deg2rad(np.arange(0, 180))
    rhos = np.arange(-diagonal, diagonal)
    
    hough_space = np.zeros((2 * diagonal, len(thetas)))

    y_idxs, x_idxs = np.nonzero(image > 150)
    
    for i in range(len(x_idxs)):
        x = x_idxs[i]
        y = y_idxs[i]
        
        for theta_idx in range(len(thetas)):
            theta = thetas[theta_idx]
            rho = int(x * np.cos(theta) + y * np.sin(theta))
            hough_space[rho + diagonal, theta_idx] += 1

    n_mayores = 10
    indices_aplanados = np.argpartition(hough_space.flatten(), -n_mayores)[-n_mayores:]
    valores_mayores = hough_space.flatten()[indices_aplanados]
    indices_mayores = np.unravel_index(indices_aplanados, hough_space.shape)

    orden = np.argsort(-valores_mayores)
    valores_mayores = valores_mayores[orden]
    indices_mayores = tuple(indice[orden] for indice in indices_mayores)

    lista = []

    for i in range(len(indices_mayores[0])):
        lista.append((rhos[indices_mayores[0][i]], np.rad2deg(thetas[indices_mayores[1][i]])))

    return lista

In [7]:
matrix = houghMatrix(uniformImage)

In [8]:
def get_line_points(rho, theta, width, height):
    theta_rad = np.deg2rad(theta)
    
    # Calcular el coseno y seno de theta
    a = np.cos(theta_rad)
    b = np.sin(theta_rad)
    
    # Calcular los puntos de intersección con los bordes de la imagen
    points = []
    
    # Intersección con el borde superior (y = 0)
    if b != 0:
        x_top = int(rho / a)
        if 0 <= x_top <= width:
            points.append((x_top, 0))
    
    # Intersección con el borde inferior (y = height)
    if b != 0:
        #x_bottom = int((rho - height * b) / a)
        x_bottom = int(((-b/a)*height) + (rho/a))
        if 0 <= x_bottom <= width:
            points.append((x_bottom, height))
    
    # Intersección con el borde izquierdo (x = 0)
    if a != 0:
        y_left = int(rho / b)
        if 0 <= y_left <= height:
            points.append((0, y_left))
    
    # Intersección con el borde derecho (x = width)
    if a != 0:
        #y_right = int((rho - width * a) / b)
        y_right = int(((-a/b)*width) + (rho/b))
        if 0 <= y_right <= height:
            points.append((width, y_right))

    return points

In [9]:
height, width = uniformImage.shape

color_image = cv2.cvtColor(uniformImage, cv2.COLOR_GRAY2BGR)


for rho, theta in matrix:

    points = get_line_points(rho, theta, width, height)

    if len(points) == 2:
        pt1, pt2 = points
        # Dibujar la línea en rojo
        color = (0, 0, 255)  # Rojo en formato BGR
        cv2.line(color_image, pt1, pt2, color, 2)

cv2.imwrite(textImage + 'RedLine.png', color_image)

True

In [10]:
def printPixels(image):
    width, height = len(image[0]), len(image)

    lista = []

    for y in range(1, height-1):
        for x in range(1, width-1):
            #print(image[y][x])
            lista.append(image[y][x])
    
    return lista